# Stock optimization using PuLP
This notebook's aim is trying to create a portfolio of stocks from historical stocks data. To know more, check the README.
First of all, we'd like to add the required libraries: we'll need PuLP, a Linear Programming library for Python, and Pandas to manage the dataset.

In [1]:
!pip3 install pandas pulp
import pandas as pd
from pulp import *

In [2]:
!pip install tqdm
from tqdm import tqdm

In [3]:
budget = 1000

## Data retrieval
We will first try to use the data from the [S&P 500 stocks](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), for which the historical prices [can be found on Kaggle](https://www.kaggle.com/camnugent/sandp500). Let's therefore start by importing these data.

In [4]:
historical_data = pd.read_csv('all_stocks_5yr.csv')
historical_data = historical_data.dropna()

# Solving the first problem: optimizing for a buy in the first day and sell on the last
The simplest problem to solve would be the following: given our budget of 1000$, which stocks should we buy for a long-term 5 year investment, in which we can only sell on the last day? This will serve more as a **toy problem** to experiment with PuLP: the solution will obviously just be the maximum possible quantity of stocks of the company with the highest difference between opening and closing.

In [5]:
long_term = LpProblem("Long-term_investment", LpMaximize)

## Variables
To do this, we'll need to throw away lots of data: we're just interested in the buy prices (opening price for a stock on the first day, i.e. 2013-02-08) and the sell prices (closing price on the last day, i.e. 2018-02-07). As stated in the README, we - for now - only have one constraint: we must spend less than our budget.
Let's start to define the variables: we'll want one variable per available stock.

In [6]:
# Let's start by saving our stock symbols into a list
symbols = historical_data["Name"].unique()
# and create a DataFrame to store the differences in
prices = pd.DataFrame(columns=["Name", "Difference"])
# Then, for each of them, we take the first and last row
for firm in symbols:
    opening = historical_data[historical_data["Name"]==firm].iloc[0]["open"]
    closing = historical_data[historical_data["Name"]==firm].iloc[-1]["close"]
    if not opening != opening and not closing != closing: # This is a check for nans
        prices = prices.append({"Name": firm, "InitialPrice":opening, "Difference": closing-opening}, ignore_index=True)
prices

,Name,Difference,InitialPrice
0,AAL,36.3300,15.0700
1,AAPL,91.8258,67.7142
2,AAP,31.5900,78.3400
3,ABBV,77.2500,36.3700
4,ABC,47.7000,46.5200
...,...,...,...
500,XYL,44.7800,27.0100
501,YUM,16.4300,63.7000
502,ZBH,45.7600,75.0200
503,ZION,29.9900,24.0300


In [7]:
# Then, for each difference we'll want to create a PuLP variable, with no upper/lower bounds, continuous: basically, the default
variables = [LpVariable(firm) for firm in prices["Name"]]

## Objective function
The objective function is pretty simple right now: we'll just have to sum the products of the difference between the opening and closing prices by the bought/sold quantity.

In [8]:
# Now, the objective function
long_term += lpSum([prices.iloc[i]["Difference"] * variables[i] for i in range(len(prices["Name"]))])

## Constraints
Finally, our constraint: we have a 1000$ budget.

In [9]:
long_term += (lpSum([prices.iloc[i]["InitialPrice"] * variables[i] for i in range(len(prices["Name"]))])) <= 1000.00

## Solution
A final check of everything, then let's make the magic happen!

In [10]:
print(long_term)

Long-term_investment:
MAXIMIZE
22.990000000000002*A + 36.33*AAL + 31.590000000000003*AAP + 91.82579999999999*AAPL + 77.25*ABBV + 47.699999999999996*ABC + 24.28*ABT + 82.14*ACN + 154.03*ADBE + 40.629999999999995*ADI + 11.879999999999999*ADM + 52.85*ADP + 92.87*ADS + 71.75*ADSK + 20.4*AEE + 19.900000000000006*AEP + -0.6399999999999988*AES + 131.66000000000003*AET + 35.21999999999999*AFL + 83.12999999999998*AGN + 21.71*AIG + 11.810000000000002*AIV + 49.94*AIZ + 28.11*AJG + 28.980000000000004*AKAM + 44.849999999999994*ALB + 201.93*ALGN + 40.235*ALK + 51.82000000000001*ALL + 30.209999999999994*ALLE + 24.040000000000006*ALXN + 35.28*AMAT + 9.02*AMD + 32.94*AME + 41.03*AMG + 91.86*AMGN + 93.90000000000002*AMP + 64.24*AMT + 1155.38*AMZN + 48.160000000000004*ANDV + 80.31*ANSS + 172.45999999999998*ANTM + 84.54000000000002*AON + 45.3375*AOS + -44.58*APA + -25.409999999999997*APC + 70.66000000000001*APD + 53.52*APH + 4.799999999999997*APTV + 46.239999999999995*ARE + -1.6600000000000001*ARNC + 56.2

In [11]:
status = long_term.solve()
print(LpStatus[status])

Unbounded


In [12]:
# Each of the variables is printed with it's resolved optimum value
for v in long_term.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

NVDA = 80.840744


## Nice!
Everything went as expected: the solver returned the stock with the highest difference, with the quantity to buy. This is of course not useful in any *financial* way, but helped me better understand the library and the problem.
Now, we can scale up things a bit.
# Introducing the time dimension: we can now open positions every day.
Now, we'd like the variables to represent having a stock for a given day, not the stock itself. We'll consider a positive value as *call stock XXX on day YYY*, while a negative as a puts. For now, we'll consider positions to only be closed on the last day: what is called/put, stays open until the last day. 
## Expected results
The results will probably be simple: the stock/day couple with the maximum difference from the last day will be the only stock being $> 0$.

In [13]:
always_open = LpProblem("OpenWhenPossibleCloseOnLastDay", LpMaximize)


We now want to add two columns: an `index` column, containing an identifier for a given stock in a given day, and a `difference` column, stating the price different between that day open and the last day's close.

In [14]:
historical_data["index"] = historical_data.Name+historical_data.date
historical_data

,date,open,high,low,close,volume,Name,index
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,AAL2013-02-08
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,AAL2013-02-11
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,AAL2013-02-12
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,AAL2013-02-13
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,AAL2013-02-14
...,...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS,ZTS2018-02-01
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS,ZTS2018-02-02
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS,ZTS2018-02-05
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS,ZTS2018-02-06


In [15]:
for symbol in tqdm(symbols):
    close = historical_data[historical_data["Name"]==symbol].iloc[-1]["close"]
    historical_data.loc[historical_data.Name==symbol,"Difference"] = close - historical_data.loc[historical_data.Name==symbol, "open"]

100%|██████████| 505/505 [00:46<00:00, 10.95it/s]


In [16]:
historical_data

,date,open,high,low,close,volume,Name,index,Difference
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,AAL2013-02-08,36.33
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,AAL2013-02-11,36.51
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,AAL2013-02-12,36.95
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,AAL2013-02-13,37.10
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,AAL2013-02-14,36.46
...,...,...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS,ZTS2018-02-01,-2.98
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS,ZTS2018-02-02,-3.67
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS,ZTS2018-02-05,-2.78
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS,ZTS2018-02-06,1.12


Now, for each row we'll create a PuLP variable then define the objective function

In [17]:
# Then, for each difference we'll want to create a PuLP variable, with no upper/lower bounds, continuous: basically, the default
always_open_variables = [LpVariable(firm) for firm in tqdm(historical_data["index"])]
print("Created the variables...\nNow creating the objective function")

always_open += lpSum([historical_data.iloc[i]["Difference"] * always_open_variables[i] for i in tqdm(range(len(historical_data["index"])))])
print("Created the objective function...\nNow creating the constraints")
always_open += (lpSum([historical_data.iloc[i]["open"] * always_open_variables[i] for i in tqdm(range(len(historical_data["index"])))])) <= 1000.00


  0%|          | 659/619029 [00:00<01:33, 6589.08it/s]

Created the variables...
Now creating the objective function


  0%|          | 458/619029 [00:00<02:15, 4573.73it/s]

Created the objective function...
Now creating the constraints


100%|██████████| 619029/619029 [01:26<00:00, 7132.66it/s]


In [18]:
status = always_open.solve()
print(LpStatus[status])

Unbounded


In [19]:
# Each of the variables is printed with it's resolved optimum value
for v in always_open.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

RCL2013_04_05 = 31.897927


As we expected, the only bought stock was the *Royal Caribbean* on May the 5th, 2013.

# Day trading: buy at open, sell at close
We first have to add a `DayDifference` stating our profit per stock if we buy at open and sell at close. I'd expect the outcome to be the maximum purchasable quantity of the most increasing stock for each day. Nothing that interesting but still a small leap forward. We'll use a reduced dataset since now it is computationally heavier.

In [20]:
historical_data_simplified = historical_data.head(10000)

In [21]:
historical_data_simplified["DayDifference"] = historical_data_simplified["close"]- historical_data_simplified["open"]
historical_data_simplified

<ipython-input-21-28b44fe4bc70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_data_simplified["DayDifference"] = historical_data_simplified["close"]- historical_data_simplified["open"]


,date,open,high,low,close,volume,Name,index,Difference,DayDifference
0,2013-02-08,15.07,15.12,14.630,14.75,8407500,AAL,AAL2013-02-08,36.33,-0.32
1,2013-02-11,14.89,15.01,14.260,14.46,8882000,AAL,AAL2013-02-11,36.51,-0.43
2,2013-02-12,14.45,14.51,14.100,14.27,8126000,AAL,AAL2013-02-12,36.95,-0.18
3,2013-02-13,14.30,14.94,14.250,14.66,10259500,AAL,AAL2013-02-13,37.10,0.36
4,2013-02-14,14.94,14.96,13.160,13.99,31879900,AAL,AAL2013-02-14,36.46,-0.95
...,...,...,...,...,...,...,...,...,...,...
9995,2017-10-18,150.33,153.43,149.030,153.00,4290947,ADBE,ADBE2017-10-18,42.01,2.67
9996,2017-10-19,165.50,172.15,164.420,171.73,13522642,ADBE,ADBE2017-10-19,26.84,6.23
9997,2017-10-20,171.50,175.87,171.355,175.64,6019412,ADBE,ADBE2017-10-20,20.84,4.14
9998,2017-10-23,175.68,175.85,171.830,172.16,3220814,ADBE,ADBE2017-10-23,16.66,-3.52


In [30]:
day_trading = LpProblem("DayTrading", LpMaximize)

Now, let's think a bit about the problem: we now have a different cash availability each day. 
We'll want to constrain every day's expense to what was earned the day before: $\sum_i^{n} \textrm{stock_open}_i * \textrm{stock_var}_i - \sum_j^n \textrm{stock_close}_{yesterday,i}*\textrm{stock_var}_{yesterday,i} \le 0$.
We'll also need to specify that on the first day, we want to respect our budget: $\sum_i^{n} \textrm{stock_open}_i * \textrm{stock_var}_i \le 1000$

In [31]:
day_trading_stock_vars = {index.replace("-", "_"):LpVariable(index) for index in tqdm(historical_data_simplified["index"])} # The replace is needed because PuLP does the same
date_list = [date.replace("-", "_") for date in historical_data_simplified["date"].unique()]

100%|██████████| 10000/10000 [00:00<00:00, 290816.71it/s]


In [32]:
day_trading += (lpSum([historical_data_simplified[(historical_data_simplified["date"]=="2013-02-08".replace("_", "-"))&(historical_data_simplified["Name"]==firm)]["open"]*(day_trading_stock_vars[firm+"2013_02_08"]) for firm in historical_data_simplified["Name"].unique()])) <= 1000
for i in tqdm(range(1, len(date_list)-1)):
    date = str(date_list[i])
    prevDate = str(date_list[i-1])
    day_trading += (lpSum([historical_data_simplified[(historical_data_simplified["date"]==date.replace("_", "-"))&(historical_data_simplified["Name"]==firm)]["open"]*(day_trading_stock_vars[firm+date] if firm+date in day_trading_stock_vars else 0) for firm in historical_data_simplified["Name"].unique()])) - (lpSum([historical_data_simplified[(historical_data_simplified["date"]==prevDate.replace("_", "-"))&(historical_data_simplified["Name"]==firm)]["close"]*(day_trading_stock_vars[firm+prevDate] if firm+prevDate in day_trading_stock_vars else 0) for firm in historical_data_simplified["Name"].unique()]))<=0

100%|██████████| 1257/1257 [00:35<00:00, 35.13it/s]


Finally, the **objective function**: we'll want to maximize the last day's money.

In [33]:
lastDay = str(date_list[i])
day_trading += lpSum([historical_data_simplified[(historical_data_simplified["date"]==lastDay.replace("_", "-"))&(historical_data_simplified["Name"]==firm)]["close"]*(day_trading_stock_vars[firm+date] if firm+date in day_trading_stock_vars else 0) for firm in historical_data_simplified["Name"].unique()])

In [34]:
print(day_trading)

DayTrading:
MAXIMIZE
51.18*AAL2018_02_06 + 112.2*AAP2018_02_06 + 163.03*AAPL2018_02_06 + 111.2*ABBV2018_02_06 + 91.54*ABC2018_02_06 + 58.86*ABT2018_02_06 + 154.69*ACN2018_02_06 + 0.0
SUBJECT TO
_C1: 15.07 AAL2013_02_08 + 78.34 AAP2013_02_08 + 67.7142 AAPL2013_02_08
 + 36.37 ABBV2013_02_08 + 46.52 ABC2013_02_08 + 34.39 ABT2013_02_08
 + 73.01 ACN2013_02_08 + 38.31 ADBE2013_02_08 <= 1000

_C2: - 14.75 AAL2013_02_08 + 14.89 AAL2013_02_11 - 78.9 AAP2013_02_08
 + 78.65 AAP2013_02_11 - 67.8542 AAPL2013_02_08 + 68.0714 AAPL2013_02_11
 - 36.25 ABBV2013_02_08 + 36.13 ABBV2013_02_11 - 46.89 ABC2013_02_08
 + 46.85 ABC2013_02_11 - 34.41 ABT2013_02_08 + 34.42 ABT2013_02_11
 - 73.31 ACN2013_02_08 + 73.09 ACN2013_02_11 - 39.12 ADBE2013_02_08
 + 38.99 ADBE2013_02_11 <= 0

_C3: - 14.46 AAL2013_02_11 + 14.45 AAL2013_02_12 - 78.39 AAP2013_02_11
 + 78.39 AAP2013_02_12 - 68.5614 AAPL2013_02_11 + 68.5014 AAPL2013_02_12
 - 35.85 ABBV2013_02_11 + 35.77 ABBV2013_02_12 - 46.76 ABC2013_02_11
 + 46.7 ABC2013_02_12

Finally, we append the results to a `pd.DataFrame` to pretty-print and sort them.

In [36]:
day_trading.solve()
results = pd.DataFrame(columns=["name", "date", "quantity", "value"])
for v in tqdm(day_trading.variables()):
    if v.varValue > 0:
        results = results.append({"name": v.name[:-10], "date": v.name[-10:], "quantity": v.varValue, "value":float(historical_data[(historical_data["Name"]==v.name[:-10])&(historical_data["date"]==v.name[-10:].replace("_", "-"))]["close"])*v.varValue}, ignore_index=True)

100%|██████████| 9993/9993 [01:18<00:00, 127.29it/s]


In [37]:
results.sort_values("date")

,name,date,quantity,value
837,ABC,2013_02_08,21.496131,1.007954e+03
838,ABC,2013_02_11,21.514484,1.006017e+03
1057,ACN,2013_02_12,13.801856,1.012642e+03
0,AAL,2013_02_13,70.814137,1.038135e+03
1058,ACN,2013_02_14,14.180238,1.037001e+03
...,...,...,...,...
294,AAL,2018_01_31,303322.620000,1.647648e+07
558,AAP,2018_02_01,141745.400000,1.662532e+07
295,AAL,2018_02_02,310811.690000,1.619329e+07
707,AAPL,2018_02_05,101780.570000,1.592764e+07


# Finally: open and close positions every day
To do this, I probably need a new class of variables: something that tells us when a position is closed. A good idea might be having binary variables multiplied by the positions and their inverse multiplied by the initial buying price to get the money we have in the bank.